# Laboratorio 2: TNG

## Analizamos el dataset Twenty News Groups, aplicando Naive Bayes de la librería SkLearn y respondiendo el siguiente cuestionario de referencia:

1- ¿Cuántos artículos tiene el train set de TNG?

2- ¿A qué categoría pertenece el artículo cuyo índice es [5]?

3- Indique cuál de las siguientes palabras se encuentra dentro de la lista de stopwords utilizadas.

4- Escriba en minúsculas qué da como resultado la lemmatización de la palabra 'being'.

5- Escriba en minúsculas qué da como resultado el stemming de la palabra 'president'.

6- ¿Cuál es el tamaño del vocabulario si luego de procesar los artículos según la notebook se descartan las palabras que aparecen en más del 60% de los artículos y aparecen por lo menos 10 veces?

7- ¿A qué índice corresponde el stemming de la palabra 'president' dentro del vocabulario?

8- ¿Cuál es el accuracy para el train set del modelo de Naive Bayes Multinomial entrenado con alpha=3?

9- ¿Cuál es la cantidad de artículos en el test set del dataset de Twenty News Group?

10-¿Cuál es el accuracy para el test set del modelo de Naive Bayes Multinomial entrenado anteriormente?




In [1]:
#Descargamos el dataset TNG para analizarlo
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=False)

# Respuesta1:

In [2]:
len(twenty_train.data) #Cantidad de artículos periodísticos

11314

# Respuesta2:

In [3]:
twenty_train["target"][5] #Lista que indica a que clase pertenece los artículos
twenty_train["target_names"][4] #Descripción de las clases

print(twenty_train["target_names"][twenty_train["target"][5]]) #Categoría que pertenece al artículo cuyo índice es [5]

comp.sys.mac.hardware




Vamos a aplicar el siguiente procesamiento utilizando los conceptos vistos en clase:

    Tokenization (nltk)
    Lemmatization (nltk)
    Stop Words (nltk)
    Stemming (nltk)
    Filtrado de palabras
    Obtención del vocabulario (countvectorizer)
    Transformación de los artículos en vectores
    Armado del modelo de Naive Bayes Multinomial
    Evaluación con el Train Set
    Evaluación con el Test Set



In [4]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
#Procesamos todos los artículos:
articulos_procesados=list()
for idx in range(len(twenty_train.data)):
    if idx%1000==0:
        print(f'se procesaron {idx} artículos')
    art=twenty_train.data[idx]
    tok=word_tokenize(art)
    lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
    stop = [x for x in lem if x not in stopwords.words('english')]
    stem=[stemmer.stem(x) for x in stop]
    alpha=[x for x in stem if x.isalpha()]
    articulos_procesados.append(" ".join(alpha))

se procesaron 0 artículos
se procesaron 1000 artículos
se procesaron 2000 artículos
se procesaron 3000 artículos
se procesaron 4000 artículos
se procesaron 5000 artículos
se procesaron 6000 artículos
se procesaron 7000 artículos
se procesaron 8000 artículos
se procesaron 9000 artículos
se procesaron 10000 artículos
se procesaron 11000 artículos


# Respuesta3:

In [6]:
#Buscamos cual de estas 3 palabras se encuentran dentro de stopwords:
print("themselves" in stopwords.words('english'))
print("always" in stopwords.words('english'))
print("anything" in stopwords.words('english'))
#La palabra "themselves" se encuentra dentro de la lista de stopwords utilizadas

True
False
False


# Respuesta4:

In [7]:
lemmatizer.lemmatize('being',pos='v')  # Lemmatizamos la palabra "being"

'be'

# Respuesta5:

In [8]:
stemmer.stem('president')  #Buscamos el stemming de la palabra 'president'.

'presid'

Vamos a continuar cargando codigo para seguir con las respuestas

In [9]:
# Importamos CountVectorizer para procesar las palabras:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_df=0.6,min_df=10) #  Paso los parametros 
count_vect.fit(articulos_procesados) #Aprende el vocabulario y le asigna un código a cada palabra

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.6, max_features=None, min_df=10,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

# Respuesta6:

In [10]:
len(count_vect.vocabulary_)   #Tamaño del vocabulario

9139

# Respuesta 7:

In [11]:
count_vect.vocabulary_['presid']  #Índice del stemming de la palabra 'president' dentro del vocabulario

6255

# Respuesta 8:
### Evaluación con el Train Set

In [12]:
X_train_data= count_vect.transform(articulos_procesados)  #Se arma el vector de frecuencias de palabras por articulo

In [13]:
from sklearn.naive_bayes import MultinomialNB            #Implementamos un clasificador Naive Bayes discreto
clf = MultinomialNB(alpha=3.0)                           #Instanciamos la clase y utilizamos smoothing laplaciano con alpha=3.
clf.fit(X_train_data.toarray(), twenty_train["target"])  #Entrenamos nuestro modelo

MultinomialNB(alpha=3.0, class_prior=None, fit_prior=True)

In [14]:
#Calculamos el Accuracy de train
Accuracy_Train_Set=clf.score(X_train_data.toarray(), twenty_train["target"])
print(Accuracy_Train_Set)

0.9129397207000177


# Respuesta 9:
### Evaluación con el Train Set

In [15]:
#Descargamos el dataset TNG para test
twenty_test = fetch_20newsgroups(subset='test', shuffle=False)

In [16]:
len(twenty_test.data)  #cantidad de artículos en el test set

7532

# Respuesta 10:

In [17]:
#Procesamos todos los artículos para test:
articulos_procesados_test=list()
for idx in range(len(twenty_test.data)):
    if idx%1000==0:
        print(f'se procesaron {idx} artículos')
    art=twenty_test.data[idx]
    tok=word_tokenize(art)
    lem=[lemmatizer.lemmatize(x,pos='v') for x in tok]
    stop = [x for x in lem if x not in stopwords.words('english')]
    stem=[stemmer.stem(x) for x in stop]
    alpha=[x for x in stem if x.isalpha()]
    articulos_procesados_test.append(" ".join(alpha))

se procesaron 0 artículos
se procesaron 1000 artículos
se procesaron 2000 artículos
se procesaron 3000 artículos
se procesaron 4000 artículos
se procesaron 5000 artículos
se procesaron 6000 artículos
se procesaron 7000 artículos


In [18]:
#Transformamos los artículos de test procesados
X_test_data= count_vect.transform(articulos_procesados_test)

In [19]:
#Calculamos el score de test con el modelo entrenado para train
clf.score(X_test_data.toarray(), twenty_test["target"]) 

0.7841210833775889